# Лабораторная работа №2
## Оценки математического ожидания, дисперсии, медианы и моделирование совместных распределений

**Дисциплина:** Статистика и машинное обучение


## Импорт необходимых библиотек


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma, expon
import pandas as pd
import seaborn as sns

# Настройка для красивого отображения
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
sns.set_style("whitegrid")

# Для воспроизводимости результатов
np.random.seed(42)


---

## Задание 1.1: Гамма-распределение

Пусть случайная величина $\xi$ имеет распределение, задаваемое плотностью:

$$f_{\xi}(x) = \theta^2 x e^{-\theta x}, \quad x \geq 0$$

Для каждого $\theta \in \{0.5, 2, 8\}$:

**(a)** Аналитически вычислить математическое ожидание, дисперсию и математическое ожидание квадрата $\xi$.

**(b)** Для $k \in \{2^4, 2^5, \ldots, 2^{15}\}$ построить выборку из $k$ элементов. Для каждой из них посчитать оценки: математического ожидания, дисперсии и квадрата математического ожидания. Визуализировать результаты на графиках.


### (a) Аналитическое решение

**Идентификация распределения:**

Плотность $f_{\xi}(x) = \theta^2 x e^{-\theta x}$ соответствует **гамма-распределению** с параметрами:
- Параметр формы: $\alpha = 2$
- Параметр скорости (rate): $\lambda = \theta$

Общая формула гамма-распределения:
$$f(x; \alpha, \lambda) = \frac{\lambda^{\alpha}}{\Gamma(\alpha)} x^{\alpha-1} e^{-\lambda x}, \quad x \geq 0$$

При $\alpha = 2$ и $\lambda = \theta$ получаем:
$$f(x) = \frac{\theta^2}{\Gamma(2)} x^{2-1} e^{-\theta x} = \theta^2 x e^{-\theta x}$$

что совпадает с исходной плотностью.

**Аналитические характеристики:**

Для гамма-распределения $\Gamma(\alpha, \lambda)$ справедливы формулы:

$$E[\xi] = \frac{\alpha}{\lambda} = \frac{2}{\theta}$$

$$D[\xi] = \frac{\alpha}{\lambda^2} = \frac{2}{\theta^2}$$

$$E[\xi^2] = D[\xi] + (E[\xi])^2 = \frac{2}{\theta^2} + \frac{4}{\theta^2} = \frac{6}{\theta^2}$$

$$(E[\xi])^2 = \frac{4}{\theta^2}$$


In [ ]:
# Параметры theta
thetas = [0.5, 2, 8]

# Функция для вычисления аналитических значений
def analytical_values(theta):
    E_xi = 2 / theta
    D_xi = 2 / (theta**2)
    E_xi_sq = 6 / (theta**2)
    E_xi_squared = (2 / theta)**2
    return E_xi, D_xi, E_xi_sq, E_xi_squared

# Создаем таблицу аналитических значений
results_table = []
for theta in thetas:
    E_xi, D_xi, E_xi_sq, E_xi_sq_val = analytical_values(theta)
    results_table.append({
        'θ': theta,
        'E[ξ]': f"{E_xi:.6f}",
        'D[ξ]': f"{D_xi:.6f}",
        'E[ξ²]': f"{E_xi_sq:.6f}",
        '(E[ξ])²': f"{E_xi_sq_val:.6f}"
    })

df_analytical = pd.DataFrame(results_table)
print("Аналитические значения для различных θ:")
print("="*80)
print(df_analytical.to_string(index=False))
print("="*80)


### (b) Моделирование и визуализация оценок


In [ ]:
# Размеры выборок: k ∈ {2^4, 2^5, ..., 2^15}
k_values = [2**i for i in range(4, 16)]

# Функция для вычисления оценок по выборке
def compute_estimates(sample):
    E_hat = np.mean(sample)
    D_hat = np.var(sample, ddof=0)  # Смещенная оценка дисперсии
    E_sq_hat = np.mean(sample**2)
    E_squared_hat = (np.mean(sample))**2
    return E_hat, D_hat, E_sq_hat, E_squared_hat

# Генерация выборок и вычисление оценок для каждого theta
results = {theta: {'k': [], 'E_hat': [], 'D_hat': [], 'E_sq_hat': [], 'E_squared_hat': []} 
            for theta in thetas}

for theta in thetas:
    # Параметры гамма-распределения: shape=2, scale=1/theta
    dist = gamma(a=2, scale=1/theta)
    
    for k in k_values:
        sample = dist.rvs(size=k)
        E_hat, D_hat, E_sq_hat, E_squared_hat = compute_estimates(sample)
        
        results[theta]['k'].append(k)
        results[theta]['E_hat'].append(E_hat)
        results[theta]['D_hat'].append(D_hat)
        results[theta]['E_sq_hat'].append(E_sq_hat)
        results[theta]['E_squared_hat'].append(E_squared_hat)


In [ ]:
# Построение графиков для каждого theta
for theta in thetas:
    E_xi, D_xi, E_xi_sq, E_xi_sq_val = analytical_values(theta)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Оценки для $\\theta = {theta}$', fontsize=16, fontweight='bold')
    
    k_vals = results[theta]['k']
    
    # График 1: Оценка математического ожидания
    ax1 = axes[0, 0]
    ax1.plot(k_vals, results[theta]['E_hat'], 'b-o', markersize=4, linewidth=1.5, label='Оценка $\\hat{E}[\\xi]$')
    ax1.axhline(y=E_xi, color='r', linestyle='--', linewidth=2, label=f'Теоретическое $E[\\xi] = {E_xi:.4f}$')
    ax1.set_xlabel('Размер выборки $k$', fontsize=12)
    ax1.set_ylabel('Оценка $\\hat{E}[\\xi]$', fontsize=12)
    ax1.set_title('Оценка математического ожидания', fontsize=13, fontweight='bold')
    ax1.set_xscale('log', base=2)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)
    
    # График 2: Оценка дисперсии
    ax2 = axes[0, 1]
    ax2.plot(k_vals, results[theta]['D_hat'], 'g-o', markersize=4, linewidth=1.5, label='Оценка $\\hat{D}[\\xi]$')
    ax2.axhline(y=D_xi, color='r', linestyle='--', linewidth=2, label=f'Теоретическое $D[\\xi] = {D_xi:.4f}$')
    ax2.set_xlabel('Размер выборки $k$', fontsize=12)
    ax2.set_ylabel('Оценка $\\hat{D}[\\xi]$', fontsize=12)
    ax2.set_title('Оценка дисперсии', fontsize=13, fontweight='bold')
    ax2.set_xscale('log', base=2)
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)
    
    # График 3: Оценка E[xi^2]
    ax3 = axes[1, 0]
    ax3.plot(k_vals, results[theta]['E_sq_hat'], 'm-o', markersize=4, linewidth=1.5, label='Оценка $\\hat{E}[\\xi^2]$')
    ax3.axhline(y=E_xi_sq, color='r', linestyle='--', linewidth=2, label=f'Теоретическое $E[\\xi^2] = {E_xi_sq:.4f}$')
    ax3.set_xlabel('Размер выборки $k$', fontsize=12)
    ax3.set_ylabel('Оценка $\\hat{E}[\\xi^2]$', fontsize=12)
    ax3.set_title('Оценка $E[\\xi^2]$', fontsize=13, fontweight='bold')
    ax3.set_xscale('log', base=2)
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=10)
    
    # График 4: Оценка (E[xi])^2
    ax4 = axes[1, 1]
    ax4.plot(k_vals, results[theta]['E_squared_hat'], 'c-o', markersize=4, linewidth=1.5, label='Оценка $(\\hat{E}[\\xi])^2$')
    ax4.axhline(y=E_xi_sq_val, color='r', linestyle='--', linewidth=2, label=f'Теоретическое $(E[\\xi])^2 = {E_xi_sq_val:.4f}$')
    ax4.set_xlabel('Размер выборки $k$', fontsize=12)
    ax4.set_ylabel('Оценка $(\\hat{E}[\\xi])^2$', fontsize=12)
    ax4.set_title('Оценка $(E[\\xi])^2$', fontsize=13, fontweight='bold')
    ax4.set_xscale('log', base=2)
    ax4.grid(True, alpha=0.3)
    ax4.legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()


**Выводы:**
- Все оценки с ростом размера выборки $k$ сходятся к своим аналитическим значениям.
- При малых значениях $\theta$ (например, $\theta = 0.5$) наблюдается больший разброс оценок.
- При больших значениях $\theta$ (например, $\theta = 8$) оценки сходятся быстрее и более стабильны.
- Это соответствует закону больших чисел: с увеличением размера выборки выборочные оценки приближаются к теоретическим значениям.


---

## Задание 1.2: Сдвинутое экспоненциальное распределение

Дана плотность распределения случайной величины $\xi$:

$$f_{\xi}^{\lambda, a}(x) = \begin{cases} \lambda e^{-\lambda(x-a)}, & x \geq a \\ 0, & \text{иначе} \end{cases}$$

Пусть $(\lambda, a) = (2, 2)$.

**(a)** Аналитически вычислить значение моды, математического ожидания и медианы.

**(b)** Создать две выборки: большую (10000 наблюдений) и маленькую (20 наблюдений). Построить оценки моды, математического ожидания и медианы.

**(c)** Построить для каждой выборки гистограмму с вертикальными линиями оценок и график плотности с аналитическими значениями.

**(d)** Исследовать сходимость медианы и математического ожидания при изменении размера выборки.


### (a) Аналитическое решение

Данное распределение является **сдвинутым экспоненциальным распределением**:

$$\xi = a + \eta, \quad \text{где } \eta \sim \text{Exp}(\lambda)$$

**Мода:**

Плотность $f(x) = \lambda e^{-\lambda(x-a)}$ строго убывает на $[a, \infty)$, так как её производная отрицательна:
$$f'(x) = -\lambda^2 e^{-\lambda(x-a)} < 0$$

Следовательно, максимум плотности достигается в левой границе области определения:
$$\text{mode} = a = 2$$

**Математическое ожидание:**

Для экспоненциального распределения $E[\eta] = \frac{1}{\lambda}$, поэтому:
$$E[\xi] = a + \frac{1}{\lambda} = 2 + \frac{1}{2} = 2.5$$

**Медиана:**

Медиана $m$ определяется из условия $P(\xi \leq m) = 0.5$:

$$P(\xi \leq m) = 1 - e^{-\lambda(m-a)} = 0.5$$

$$e^{-\lambda(m-a)} = 0.5$$

$$-\lambda(m-a) = \ln(0.5) = -\ln(2)$$

$$m = a + \frac{\ln(2)}{\lambda} = 2 + \frac{\ln(2)}{2} \approx 2.3466$$


In [ ]:
# Параметры распределения
lambda_val = 2
a = 2

# Аналитические значения
mode_analytical = a
E_analytical = a + 1/lambda_val
median_analytical = a + np.log(2)/lambda_val

print("Аналитические значения:")
print(f"Мода: {mode_analytical}")
print(f"Математическое ожидание: {E_analytical}")
print(f"Медиана: {median_analytical:.6f}")


### (b) Генерация выборок и вычисление оценок


In [ ]:
# Генерация выборок
n_large = 10000
n_small = 20

sample_large = a + expon.rvs(scale=1/lambda_val, size=n_large)
sample_small = a + expon.rvs(scale=1/lambda_val, size=n_small)

# Функция для оценки моды через гистограмму
def estimate_mode(sample, bins=30):
    hist, edges = np.histogram(sample, bins=bins)
    idx = np.argmax(hist)
    return (edges[idx] + edges[idx+1]) / 2

# Функция для вычисления всех оценок
def compute_estimates_shifted_exp(sample):
    mode_est = estimate_mode(sample)
    E_est = np.mean(sample)
    median_est = np.median(sample)
    return mode_est, E_est, median_est

# Вычисление оценок
mode_large, E_large, median_large = compute_estimates_shifted_exp(sample_large)
mode_small, E_small, median_small = compute_estimates_shifted_exp(sample_small)

# Создаем таблицу сравнения
comparison_table = pd.DataFrame({
    'Характеристика': ['Мода', 'Математическое ожидание', 'Медиана'],
    'Аналитическое значение': [mode_analytical, E_analytical, f'{median_analytical:.6f}'],
    f'Большая выборка (n={n_large})': [f'{mode_large:.6f}', f'{E_large:.6f}', f'{median_large:.6f}'],
    f'Маленькая выборка (n={n_small})': [f'{mode_small:.6f}', f'{E_small:.6f}', f'{median_small:.6f}']
})

print("Сравнение аналитических значений и оценок:")
print("="*80)
print(comparison_table.to_string(index=False))
print("="*80)


### (c) Визуализация: гистограммы и графики плотности


In [ ]:
# Функция плотности для построения графика
def pdf_shifted_exp(x, lambda_val, a):
    return lambda_val * np.exp(-lambda_val * (x - a)) * (x >= a)

# Построение графиков для большой выборки
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# График 1: Гистограмма с оценками
ax1 = axes[0]
ax1.hist(sample_large, bins=50, density=True, alpha=0.7, color='skyblue', 
         edgecolor='black', label='Гистограмма')
ax1.axvline(mode_large, color='red', linestyle='--', linewidth=2, 
            label=f'Мода (оценка) = {mode_large:.4f}')
ax1.axvline(E_large, color='green', linestyle='--', linewidth=2, 
            label=f'$E[\\xi]$ (оценка) = {E_large:.4f}')
ax1.axvline(median_large, color='orange', linestyle='--', linewidth=2, 
            label=f'Медиана (оценка) = {median_large:.4f}')
ax1.set_xlabel('$x$', fontsize=12)
ax1.set_ylabel('Плотность', fontsize=12)
ax1.set_title(f'Гистограмма и оценки (n={n_large})', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# График 2: Теоретическая плотность с аналитическими значениями
ax2 = axes[1]
x_plot = np.linspace(a, a + 5, 1000)
y_plot = pdf_shifted_exp(x_plot, lambda_val, a)
ax2.plot(x_plot, y_plot, 'b-', linewidth=2, label='Плотность')
ax2.axvline(mode_analytical, color='red', linestyle='--', linewidth=2, 
            label=f'Мода = {mode_analytical}')
ax2.axvline(E_analytical, color='green', linestyle='--', linewidth=2, 
            label=f'$E[\\xi]$ = {E_analytical}')
ax2.axvline(median_analytical, color='orange', linestyle='--', linewidth=2, 
            label=f'Медиана = {median_analytical:.4f}')
ax2.set_xlabel('$x$', fontsize=12)
ax2.set_ylabel('Плотность $f(x)$', fontsize=12)
ax2.set_title('Теоретическая плотность и аналитические значения', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Построение графиков для маленькой выборки
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# График 1: Гистограмма с оценками
ax1 = axes[0]
ax1.hist(sample_small, bins=10, density=True, alpha=0.7, color='skyblue', 
         edgecolor='black', label='Гистограмма')
ax1.axvline(mode_small, color='red', linestyle='--', linewidth=2, 
            label=f'Мода (оценка) = {mode_small:.4f}')
ax1.axvline(E_small, color='green', linestyle='--', linewidth=2, 
            label=f'$E[\\xi]$ (оценка) = {E_small:.4f}')
ax1.axvline(median_small, color='orange', linestyle='--', linewidth=2, 
            label=f'Медиана (оценка) = {median_small:.4f}')
ax1.set_xlabel('$x$', fontsize=12)
ax1.set_ylabel('Плотность', fontsize=12)
ax1.set_title(f'Гистограмма и оценки (n={n_small})', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# График 2: Теоретическая плотность с аналитическими значениями
ax2 = axes[1]
x_plot = np.linspace(a, a + 5, 1000)
y_plot = pdf_shifted_exp(x_plot, lambda_val, a)
ax2.plot(x_plot, y_plot, 'b-', linewidth=2, label='Плотность')
ax2.axvline(mode_analytical, color='red', linestyle='--', linewidth=2, 
            label=f'Мода = {mode_analytical}')
ax2.axvline(E_analytical, color='green', linestyle='--', linewidth=2, 
            label=f'$E[\\xi]$ = {E_analytical}')
ax2.axvline(median_analytical, color='orange', linestyle='--', linewidth=2, 
            label=f'Медиана = {median_analytical:.4f}')
ax2.set_xlabel('$x$', fontsize=12)
ax2.set_ylabel('Плотность $f(x)$', fontsize=12)
ax2.set_title('Теоретическая плотность и аналитические значения', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### (d) Исследование сходимости медианы и математического ожидания


In [ ]:
# Различные размеры выборок
sample_sizes = [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]

medians = []
means = []

for n in sample_sizes:
    sample = a + expon.rvs(scale=1/lambda_val, size=n)
    medians.append(np.median(sample))
    means.append(np.mean(sample))

# Построение графика
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(sample_sizes, medians, 'o-', label='Медиана (оценка)', linewidth=2, markersize=6)
ax.plot(sample_sizes, means, 's-', label='Математическое ожидание (оценка)', linewidth=2, markersize=6)
ax.axhline(y=E_analytical, color='green', linestyle='--', linewidth=2, 
          label=f'Теоретическое $E[\\xi] = {E_analytical}$')
ax.axhline(y=median_analytical, color='orange', linestyle='--', linewidth=2, 
          label=f'Теоретическая медиана = {median_analytical:.4f}')
ax.set_xlabel('Размер выборки $n$', fontsize=12)
ax.set_ylabel('Значение', fontsize=12)
ax.set_title('Сходимость медианы и математического ожидания', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nВывод: Медиана сходится к своему теоретическому значению {median_analytical:.4f},")
print(f"а не к математическому ожиданию {E_analytical}. Это нормально, так как")
print(f"для сдвинутого экспоненциального распределения медиана ≠ E[ξ].")


---

## Задание 1.2: Моделирование совместного распределения двух СВ

Пусть совместное распределение двух случайных величин задано таблицей:

| $\xi \backslash \eta$ | 1 | 2 | 3 | $\ldots$ |
|---------------------|---|---|---|----------|
| $-1$ | $\frac{2}{5} \cdot \frac{1}{2^1}$ | $\frac{2}{5} \cdot \frac{1}{2^2}$ | $\frac{2}{5} \cdot \frac{1}{2^3}$ | $\ldots$ |
| $0$ | $\frac{1}{5} \cdot \frac{1}{2^1}$ | $\frac{1}{5} \cdot \frac{1}{2^2}$ | $\frac{1}{5} \cdot \frac{1}{2^3}$ | $\ldots$ |
| $1$ | $\frac{2}{5} \cdot \frac{1}{2^1}$ | $\frac{2}{5} \cdot \frac{1}{2^2}$ | $\frac{2}{5} \cdot \frac{1}{2^3}$ | $\ldots$ |

где $\eta$ принимает все значения из $\mathbb{N}$.

Вычислить корреляционную матрицу аналитически и приближенно (на основе моделирования).


### Аналитическое решение

**Совместное распределение:**

$$P(\xi = i, \eta = j) = c_i \cdot \frac{1}{2^j}$$

где $i \in \{-1, 0, 1\}$, $j \in \mathbb{N}$, и коэффициенты:

$$c_{-1} = \frac{2}{5}, \quad c_0 = \frac{1}{5}, \quad c_1 = \frac{2}{5}$$

**Маргинальные распределения:**

Для $\xi$:
$$P(\xi = i) = c_i \sum_{j=1}^{\infty} \frac{1}{2^j} = c_i \cdot 1 = c_i$$

Для $\eta$:
$$P(\eta = j) = \sum_{i} c_i \cdot \frac{1}{2^j} = \frac{1}{2^j} \sum_{i} c_i = \frac{1}{2^j}$$

**Независимость:**

Проверяем условие независимости:
$$P(\xi = i, \eta = j) = c_i \cdot \frac{1}{2^j} = P(\xi = i) \cdot P(\eta = j)$$

Условие выполнено, следовательно, $\xi$ и $\eta$ **независимы**.

**Математические ожидания:**

$$E[\xi] = (-1) \cdot \frac{2}{5} + 0 \cdot \frac{1}{5} + 1 \cdot \frac{2}{5} = 0$$

$$E[\eta] = \sum_{j=1}^{\infty} j \cdot \frac{1}{2^j} = 2$$

**Дисперсии:**

$$E[\xi^2] = (-1)^2 \cdot \frac{2}{5} + 0^2 \cdot \frac{1}{5} + 1^2 \cdot \frac{2}{5} = \frac{4}{5}$$

$$D[\xi] = E[\xi^2] - (E[\xi])^2 = \frac{4}{5} - 0 = \frac{4}{5}$$

$$E[\eta^2] = \sum_{j=1}^{\infty} j^2 \cdot \frac{1}{2^j} = 6$$

$$D[\eta] = E[\eta^2] - (E[\eta])^2 = 6 - 4 = 2$$

**Ковариация и корреляция:**

Из независимости следует:
$$\text{Cov}(\xi, \eta) = E[\xi \eta] - E[\xi]E[\eta] = 0$$

$$\rho(\xi, \eta) = \frac{\text{Cov}(\xi, \eta)}{\sqrt{D[\xi]D[\eta]}} = 0$$

**Корреляционная матрица:**

$$R = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$$


In [ ]:
# Аналитические значения
xi_values = np.array([-1, 0, 1])
xi_probs = np.array([2/5, 1/5, 2/5])

# Математические ожидания
E_xi_analytical = np.sum(xi_values * xi_probs)
E_eta_analytical = 2  # Сумма j/2^j от j=1 до бесконечности

# Дисперсии
E_xi2_analytical = np.sum(xi_values**2 * xi_probs)
D_xi_analytical = E_xi2_analytical - E_xi_analytical**2
D_eta_analytical = 2  # E[eta^2] = 6, D[eta] = 6 - 4 = 2

# Ковариация и корреляция (из независимости)
Cov_analytical = 0
Corr_analytical = 0

# Корреляционная матрица
R_analytical = np.array([[1, Corr_analytical], 
                        [Corr_analytical, 1]])

print("Аналитические значения:")
print(f"E[ξ] = {E_xi_analytical}")
print(f"E[η] = {E_eta_analytical}")
print(f"D[ξ] = {D_xi_analytical}")
print(f"D[η] = {D_eta_analytical}")
print(f"Cov(ξ,η) = {Cov_analytical}")
print(f"Corr(ξ,η) = {Corr_analytical}")
print(f"\nАналитическая корреляционная матрица:")
print(R_analytical)


### Моделирование и приближенная оценка


In [ ]:
# Размер выборки для моделирования
N = 1000000

# Генерация выборки ξ
xi_sample = np.random.choice(xi_values, size=N, p=xi_probs)

# Генерация выборки η (геометрическое распределение с p=0.5)
eta_sample = np.random.geometric(p=0.5, size=N)

# Выборочные оценки
E_xi_hat = np.mean(xi_sample)
E_eta_hat = np.mean(eta_sample)

D_xi_hat = np.var(xi_sample, ddof=1)  # Несмещенная оценка
D_eta_hat = np.var(eta_sample, ddof=1)

# Ковариация и корреляция
Cov_hat = np.cov(xi_sample, eta_sample, ddof=1)[0, 1]
Corr_hat = np.corrcoef(xi_sample, eta_sample)[0, 1]

# Корреляционная матрица
R_hat = np.corrcoef(xi_sample, eta_sample)

# Создаем таблицу сравнения
comparison_table = pd.DataFrame({
    'Величина': ['E[ξ]', 'E[η]', 'D[ξ]', 'D[η]', 'Cov(ξ,η)', 'Corr(ξ,η)'],
    'Аналитически': [E_xi_analytical, E_eta_analytical, D_xi_analytical, 
                     D_eta_analytical, Cov_analytical, Corr_analytical],
    f'Оценка (n={N})': [E_xi_hat, E_eta_hat, D_xi_hat, D_eta_hat, Cov_hat, Corr_hat]
})

print(f"Оценки на основе выборки размера {N}:")
print("="*80)
print(comparison_table.to_string(index=False))
print("="*80)

print(f"\nАналитическая корреляционная матрица:")
print(R_analytical)
print(f"\nПриближенная корреляционная матрица (на основе {N} испытаний):")
print(R_hat)


**Выводы:**
- Аналитически показано, что случайные величины $\xi$ и $\eta$ независимы, поэтому их корреляция равна нулю.
- Приближенные оценки на основе моделирования близки к аналитическим значениям.
- Небольшие отклонения от нуля в приближенной корреляции объясняются случайными флуктуациями при конечном размере выборки.
- С увеличением размера выборки приближенные оценки будут еще ближе к аналитическим значениям.
